In [1]:
from dataset import Dataset, Synthetic_data

import numpy as np
from scipy.linalg import orthogonal_procrustes

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

from tqdm import tqdm

from mu_dist import Mu
from sigma_dist import Sigma_Star
from gamma_dist import Gamma

from r_dist import R
from z_dist import Z
from phi_dist import Phi

from dataset_initialisation import GMM_Init

from scipy.stats import beta

In [2]:
μ_1 = np.array([0.5,0.25,0.25])
μ_2 = np.array([0.4,0.15,0.45])

    # [0.8018,0.2673,0.5345] 
    # [0.2673,0.8018,0.5345]
α = 2
β = 2
prior = lambda : beta.rvs(α, β)

In [3]:
from dataset_initialisation import GMM_Init

In [4]:
ds = Synthetic_data(μ_1, μ_2, prior, N_t=1000)

/Users/dariyankhan/Library/CloudStorage/OneDrive-ImperialCollegeLondon/Work (one drive)/Imperial/year_4/M4R/weighted_VI_code/spectral_clustering_weighted_VI/dataset.py:30: RuntimeWarning: invalid value encountered in divide
  self.normed_embds = self.embds / np.linalg.norm(self.embds, axis=1)[:, np.newaxis]


In [5]:
ds.gaussian_mm_init()

[[0.15444516 0.05811354 0.11310608]
 [0.29886597 0.14279986 0.2401997 ]] cluster centres
self.gamma_estimates:  [[ 0.01603768  0.00355913]
 [-0.07664768 -0.03828466]]
mu_cov 0 det: 0.0
scale_mat 0 det: 0.0016325008211448275
gamma_cov 0 det: -4.656960592089808e-22
gamma cov: [[0.00429529 0.00193915]
 [0.00193915 0.00087545]]
mu_cov 1 det: 0.0
scale_mat 1 det: 0.0016325008211448275
gamma_cov 1 det: -4.656960592089808e-22
gamma cov: [[0.00429529 0.00193915]
 [0.00193915 0.00087545]]


In [6]:
def print_attributes(obj):
    attributes = dir(obj)
    attributes = [attr for attr in attributes if not attr.startswith('__') and not callable(getattr(obj, attr))]
    for attr in attributes:
        value = getattr(obj, attr)
        print(f"{attr}: {value}")

## μ variables

In [7]:
print_attributes(ds.means_vars[0])
print()
print("cov matrix det:" , np.linalg.det(ds.means_vars[0].cov))

cov: [[0.01042869 0.00611523 0.00917748]
 [0.00611523 0.00358589 0.00538155]
 [0.00917748 0.00538155 0.00807639]]
d: 3
k: 0
mean: [0.15444516 0.05811354 0.11310608]
prior_cov: [[0.01042869 0.00611523 0.00917748]
 [0.00611523 0.00358589 0.00538155]
 [0.00917748 0.00538155 0.00807639]]
prior_mean: [0. 0. 0.]

cov matrix det: 0.0


In [8]:
print_attributes(ds.means_vars[1])
print()
print("cov matrix det:" , np.linalg.det(ds.means_vars[0].cov))

cov: [[0.01042869 0.00611523 0.00917748]
 [0.00611523 0.00358589 0.00538155]
 [0.00917748 0.00538155 0.00807639]]
d: 3
k: 1
mean: [0.29886597 0.14279986 0.2401997 ]
prior_cov: [[0.01042869 0.00611523 0.00917748]
 [0.00611523 0.00358589 0.00538155]
 [0.00917748 0.00538155 0.00807639]]
prior_mean: [0. 0. 0.]

cov matrix det: 0.0


## γ variables

In [9]:
print_attributes(ds.gamma_vars[0])
print()
print("cov matrix det:" , np.linalg.det(ds.gamma_vars[0].cov))

cov: [[0.00429529 0.00193915]
 [0.00193915 0.00087545]]
d: 3
dim: 2
k: 0
mean: [0.01603768 0.00355913]
nu: 0.005598763998511398
prior_cov: [[0.00429529 0.00193915]
 [0.00193915 0.00087545]]

cov matrix det: -4.656960592089808e-22


In [10]:
print_attributes(ds.gamma_vars[1])
print()
print("cov matrix det:" , np.linalg.det(ds.gamma_vars[1].cov))

cov: [[0.00429529 0.00193915]
 [0.00193915 0.00087545]]
d: 3
dim: 2
k: 1
mean: [-0.07664768 -0.03828466]
nu: 0.026463049828926517
prior_cov: [[0.00429529 0.00193915]
 [0.00193915 0.00087545]]

cov matrix det: -4.656960592089808e-22


## sigma star variables

In [11]:
print_attributes(ds.sigma_star_vars[0])

print("sigma_star_scale_det:", np.linalg.det(ds.sigma_star_vars[0].scale))

d: 3
dim: 2
dof: 6
first_moment: None
k: 0
nu: 0.005598763998511398
prior_dof: 6
prior_scale: [[ 0.03362751 -0.01435062]
 [-0.01435062  0.05467074]]
scale: [[ 0.02170003 -0.00094147]
 [-0.00094147  0.01760741]]
sigma_star_scale_det: 0.00038119504923230847


In [12]:
print_attributes(ds.sigma_star_vars[1])

print("sigma_star_scale_det:", np.linalg.det(ds.sigma_star_vars[1].scale))


d: 3
dim: 2
dof: 6
first_moment: None
k: 1
nu: 0.026463049828926517
prior_dof: 6
prior_scale: [[ 0.03362751 -0.01435062]
 [-0.01435062  0.05467074]]
scale: [[ 0.045555   -0.02775976]
 [-0.02775976  0.09173407]]
sigma_star_scale_det: 0.00340834052351934


In [13]:
ds.dataset_vi()

[[0.29856869 0.14348368 0.24080469]
 [0.15524466 0.05779377 0.1130133 ]] cluster centres
self.gamma_estimates:  [[-0.07687137 -0.03880978]
 [ 0.01615648  0.00313217]]
mu_cov 0 det: 0.0
scale_mat 0 det: 0.0016417689517118912
gamma_cov 0 det: -4.691439920285511e-22
gamma cov: [[0.00432709 0.00195088]
 [0.00195088 0.00087956]]
mu_cov 1 det: 0.0
scale_mat 1 det: 0.0016417689517118912
gamma_cov 1 det: -4.691439920285511e-22
gamma cov: [[0.00432709 0.00195088]
 [0.00195088 0.00087956]]


Performing VI:   0%|          | 0/1000 [00:00<?, ?it/s]

LinAlgError: Singular matrix